# Zu welchem Genre gehört das Filmposter

Ablauf:

1. Importieren der Bibliotheken
2. Laden der CSV Datei
3. Erste Datenbereinigung
4. Herunterladen der Film Poster
5. Erstellen einer neuen CSV Datei
6. 

# Importieren der benötigen Bibliotheken

In [ ]:
import logging
import os
import warnings

import matplotlib.pyplot as plt
import matplotlib.style as style
import numpy as npgc
import pandas as pd
import seaborn as sns
import tensorflow as tf
#import tensorflow_hub as hub
import shutil
import multiprocessing
import urllib.error
import urllib.request

from joblib import Parallel, delayed
from time import time


from datetime import datetime
from tensorflow.keras.preprocessing import image
from PIL import Image
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve
from tensorflow.keras import layers

print(tf.__version__)
print(tf.config.list_physical_devices())

## Laden der CSV Datei

In [ ]:
movie_data = pd.read_csv("./data/MovieGenre.csv",
                          sep=",", encoding='unicode_escape', index_col=None)
movie_data.head()

# Erste Datenbereinigung
Löschen der Zeilen die einen Null Wert enthalten, da wir mit diesen Zeilen nichts anfangen können

In [ ]:
# Entfernen von Null Werten
movie_data.dropna(subset=['imdbId', 'Genre', 'Poster'], inplace=True)

# Herunterladen der Film Poster (Parallel)
Die Filmposter, die in der CSV Datei angegeben sind werden Heruntergeladen, dies passiert parallel damit dies schneller geht.

In [ ]:
def download_parallel(movies, image_dir):
    # Create list of filenames
    filenames = movies['imdbId'].apply(lambda imbdId : os.path.join(image_dir, str(imbdId)+'.jpg'))
    # Create list of image urls
    urls = movies['Poster']

    # Create destination directory
    if os.path.exists(image_dir):
        print("Directory '{}' already exists and will be deleted.".format(image_dir))
        shutil.rmtree(image_dir)
    print("Created new directory '{}'".format(image_dir))
    os.makedirs(image_dir)

    # Define function to download one single image
    def download_image(url, filename):
        try:
            urllib.request.urlretrieve(url, filename)
            return 0
        except:
            return os.path.basename(filename).split('.')[0]

    # Download images in parallel
    start = time()
    print("\nDownloading...")
    num_cores = multiprocessing.cpu_count()
    ko_list = Parallel(n_jobs=num_cores)(delayed(download_image)(u, f) for f, u in zip(filenames, urls))

    print("\nDownload in parallel mode took %d seconds." %(time()-start))
    print("Success:", len([i for i in ko_list if i==0]))
    print("Errors:", len([i for i in ko_list if i!=0]))

    # Remove not downloaded posters from the dataframe
    print("length of Movies before removing errors: " + str(len(movies)))
    imdbIdToBeRemoved = []
    for item in ko_list:
        if(item !=0):
            imdbIdToBeRemoved.append(item)
    movies= movies[~ movies["imdbId"].astype(str).isin(imdbIdToBeRemoved)]
    print("length of Movies after removing errors: " + str(len(movies)))

    return movies


destination = './data/movie-posters'
movie_data = download_parallel(movie_data, destination)

# Erstellen einer neuen CSV Datei

In [ ]:
data_dir = "./data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
movie_data.to_csv(os.path.join(data_dir, "movies.csv"), index=False)

# Zweite Datenbereinigung
 1. Wir zählen, wie oft ein Label vorkommt.
 2. Speichern alle Labels die weniger als 1000 mal vorkommen in einer Liste
 3. Entfernen aller Labels die mit einem der Labels in der zuvor erstellten Liste übereinstimmen


In [ ]:
movie_data = pd.read_csv("./data/movies.csv")

label_freq = movie_data['Genre'].apply(lambda s: str(s).split('|')).explode().value_counts().sort_values(ascending=False)
rare = list(label_freq[label_freq<1000].index)

display(rare)
# Display what genres do we have and how many do we have
# evtl Dramen rauschmeißen
print("Number of movie posters in last download: {}\n".format(len(movie_data)))
movie_data['Genre'] = movie_data['Genre'].apply(lambda s: [l for l in str(s).split('|') if l not in rare])

movie_data.head(3)

## Input Pipeline erstellen

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(movie_data['imdbId'], movie_data['Genre'], test_size=0.2, random_state=44)
print("Number of posters for training: ", len(X_train))
print("Number of posters for validation: ", len(X_val))

# we need files instead of imdbIds
X_train = [os.path.join('./data/movie-posters/', str(f)+'.jpg') for f in X_train]
X_val = [os.path.join('./data/movie-posters/', str(f)+'.jpg') for f in X_val]
print(X_train[:3])
print(X_val[:3])

y_train = list(y_train)
y_val = list(y_val)
print(y_train[:3])

Beispiel Bilder

In [ ]:
nobs = 8 # Maximum number of images to display
ncols = 4 # Number of columns in display
nrows = nobs//ncols # Number of rows in display

style.use("default")
plt.figure(figsize=(12,4*nrows))
for i in range(nrows*ncols):
    ax = plt.subplot(nrows, ncols, i+1)
    plt.imshow(Image.open(X_train[i]))
    plt.title(y_train[i], size=10)
    plt.axis('off')

 Label encoding

In [ ]:
# Fit the multi-label binarizer on the training set
print("Labels:")
mlb = MultiLabelBinarizer()
mlb.fit(y_train)

# Loop over all labels and show them
N_LABELS = len(mlb.classes_)
for (i, label) in enumerate(mlb.classes_):
    print("{}. {}".format(i, label))

# transform the targets of the training and test sets
y_train_bin = mlb.transform(y_train)
y_val_bin = mlb.transform(y_val)

# Print example of movie posters and their binary targets
for i in range(3):
    print(X_train[i], y_train_bin[i])


## Pipeline

In [ ]:
IMG_SIZE = 224 # Height and Width of an Image
CHANNELS = 3 # 3 Stands for RGB
# 224*224*3

BATCH_SIZE = 256 # Number of learning repetitions
SHUFFLE_BUFFER_SIZE = 1024 # Shuffle the training data by a chunck of 1024 observations
AUTOTUNE = tf.data.experimental.AUTOTUNE # Adapt preprocessing and prefetching dynamically


def parse_function(filename, label):
    # Read an image from a file
    image_string = tf.io.read_file(filename)
    # Decode it into a dense vector
    image_decoded = tf.image.decode_jpeg(image_string, channels=CHANNELS)
    # Resize it to fixed shape
    image_resized = tf.image.resize(image_decoded, [IMG_SIZE, IMG_SIZE])
    # Normalize it from [0, 255] to [0.0, 1.0]
    image_normalized = image_resized / 255.0
    return image_normalized, label

def create_dataset(filenames, labels, is_training=True):
    # Create a first dataset of file paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    # Parse and preprocess observations in parallel
    dataset = dataset.map(parse_function, num_parallel_calls=AUTOTUNE)

    if is_training == True:
        # This is a small dataset, only load it once, and keep it in memory.
        dataset = dataset.cache()
        # Shuffle the data each buffer size
        dataset = dataset.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)

    # Batch the data for multiple steps
    dataset = dataset.batch(BATCH_SIZE)
    # Fetch batches in the background while the model is training.
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)

    return dataset

train_ds = create_dataset(X_train, y_train_bin)
val_ds = create_dataset(X_val, y_val_bin)

for f, l in train_ds.take(1):
    print("Shape of features array:", f.numpy().shape)
    print("Shape of labels array:", l.numpy().shape)

In [ ]:
feature_extractor_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4"
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(IMG_SIZE,IMG_SIZE,CHANNELS))
                                         #mobilenet vorstellen
feature_extractor_layer.trainable = False

In [ ]:
model = tf.kreas.Sequential([
        feature_extractor_layer,
        layers.Dense(1024, activation='relu', name='hidden_layer'),
        layers.Dense(N_LABELS, activation='sigmoid')
])

## Model bauen

In [ ]:
demoModel = tf.keras.Sequential([
    layers.Conv2D(16,(3,3),activation='relu', input_shape=(IMG_SIZE,IMG_SIZE,3)),
    layers.MaxPool2D((2,2)),
    
    layers.Conv2D(16,(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),
    
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),
    
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),
    
    layers.Conv2D(128,(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),
    
    layers.Conv2D(128,(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),
    
    layers.Flatten(),
    
    layers.Dense(128, activation='relu'),
    layers.Dense(N_LABELS, activation='sigmoid')
])

In [ ]:
model_name="complex_layers"
with open(f"./data/models/{model_name}.txt",'w') as fh:
    model.summary(print_fn=lambda x: fh.write(x + '\n'))

# Train

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-4)
loss = tf.keras.metrics.binary_crossentropy

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=["accuracy"])

EPOCHS = 30
start = time()
history = model.fit(train_ds,epochs=EPOCHS,validation_data=create_dataset(X_val, y_val_bin))

# Plot to see how good our KI is

In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, EPOCHS), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, EPOCHS), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, EPOCHS), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, EPOCHS), history.history["val_accuracy"], label="val_acc")

plt.title("Training Loss and Accuracy on Meme Classification")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(f"./data/models/{model_name}")


# Save model

In [ ]:
model_dir = "./data/models"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
model.save(f"{model_dir}/{model_name}.h5",save_format="h5")

# Load Testing Images and Prepare Them

In [ ]:
def prepare_user_images(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=CHANNELS)
    image_resized = tf.image.resize(image_decoded, [IMG_SIZE, IMG_SIZE])
    image_normalized = image_resized / 255.0
    image_normalized = np.expand_dims(image_normalized, axis= 0)
    return image_normalized, label

moviesTopPredict = []
avengers, avengersLabels = prepare_user_images("./data/validationImages/Avengers-Action-Science-Fiction.jpg",["Action","Science-Fiction"])
crazyStupidLove, crazyStupidLoveLabels = prepare_user_images("./data/validationImages/crazyStupidLove-Romanze-Komödie.jpg",["Romanze", "Komödie"])
peterchensMondfahrt, peterchensMondfahrtLabels = prepare_user_images("./data/validationImages/peterchensMondfahrt-Komödie-Fantasy-Abenteuer-Animation-Familienfilm.jpg",["Komödie", "Fantasy", "Abenteuer", "Animation", "Familienfilm"])
#Demo begleiten (Handout):
# Schritte con Datenbereinigung angefangen
# abbildung Nuronales Netz
# begriff erklären

moviesTopPredict.append((avengers, avengersLabels))
moviesTopPredict.append((crazyStupidLove, crazyStupidLoveLabels))
moviesTopPredict.append((peterchensMondfahrt, peterchensMondfahrtLabels))
movieNames = ["Avengers", "Crazy Stupid Love", "Peterchens Mondfahrt"]


# Load Model

In [ ]:
model = tf.keras.models.load_model(f"./data/models/{model_name}.h5")


for index, tuple in enumerate(moviesTopPredict):
    display("--------------"+ movieNames[index] +"------------------")
    prediction = (model.predict(tuple[0]) > 0.5).astype('int')
    prediction = pd.Series(prediction[0])
    prediction.index = mlb.classes_
    prediction = prediction[prediction==1].index.values


    display("Prediction: " + str(list(prediction)))
    display("Actual: " + str(list(tuple[1])))
